# 03 · Transfer Evaluation
Train a linear probe on stored embeddings and summarise AUROC/AUPRC.

In [ ]:
from pathlib import Path
import numpy as np
from biofm.eval.linear_probe import probe_and_report
from biofm.utils.embeddings import load_embeddings_from_disk
from biofm.utils.pipeline import load_bundle
from omegaconf import OmegaConf

cfg = OmegaConf.create({'paths': {'data_dir': 'data'}, 'project': {'seed': 7}})
image_df, rna_df = load_embeddings_from_disk(Path(cfg.paths.data_dir))
bundle = load_bundle(OmegaConf.merge(cfg, {'train': {'embedding_dim': 256}, 'data': {'select_hvg': 32, 'image_size': 128}}))
labels = {record.sample_id: record.label for record in bundle.clinical}
ids = image_df.index
features = np.hstack([image_df.loc[ids].to_numpy(), rna_df.loc[ids].to_numpy()])
label_array = np.array([labels[i] for i in ids])
_, summary = probe_and_report(features, label_array)
print(summary)